In [125]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

In [23]:
# Data Imports from src/sqldb.py
# DO NOT CHANGE BEFORE CHANGING SOURCE FILE
import os
import pandas as pd
import sqlite3

class SQLDatabase:
    """API for interacting with SQL database."""
    
    def __init__(self, db_name=None):
        if db_name is None:
            # CHANGING DEFINITION SPECIFICALLY FOR JUPYTER NOTEBOOK
            # _path_to_db = os.path.abspath("dbs/links.db")
            _path_to_db = os.path.join(
                os.path.dirname(os.getcwd()),
                "dbs",
                "links.db"
            )
        else:
            _path_to_db = db_name
        try:
            self._connection = sqlite3.connect(_path_to_db)
            self._cursor = self._connection.cursor()
        except:
            raise ValueError("Bad Connection")

    def __enter__(self):
        return self

    def __exit__(self, *args):
        self.close()

    def close(self):
        self._connection.close()

    def union_events(self, data, table):
        for item in data:
            self.add_new(item, table)
        self._connection.commit()

    def add_new(self, item, table):
        if table == "event":
            try:
                self._cursor.execute(
                    """
                    INSERT INTO event (name, link, date)
                    VALUES (?, ?, ?)
                    """, item
                )
            except sqlite3.IntegrityError:
                pass

        elif table == "pilot":
            if len(item) == 1:
                item += [""]
            try:
                self._cursor.execute(
                    """
                    INSERT INTO pilot (firstName, lastName)
                    values (?, ?)
                    """, item
                )
            except sqlite3.IntegrityError:
                pass

        elif table == "deck":
            query = """
            INSERT INTO deck (eventId, pilotId, deckUrl, name, rank)
            VALUES (?, ?, ?, ?, ?)
            """
            event_id = self._cursor.execute("""
                SELECT id
                FROM event
                WHERE link = ?
                """, (item[0], )
            ).fetchone()[0]
            pilot_id = self._cursor.execute("""
                SELECT id
                FROM pilot
                WHERE (firstName || lastName) = ?
                """, ("".join(item[1].split(maxsplit=1)),)
            ).fetchone()[0]
            items_to_insert = item[2:]
            try:
                self._cursor.execute(query, (event_id, pilot_id) + items_to_insert)
            except sqlite3.IntegrityError:
                pass

        elif table == "decklist":
            query = """
            INSERT INTO deckList (cardId, deckId, count, slot)
            VALUES (?, ?, ?, ?)
            """
            try:
                self._cursor.execute(query, item)
            except sqlite3.IntegrityError:
                pass

        elif table == "card":
            query = """
            INSERT INTO card (setNumber, setName, name, cmc, color, standardLegality, oracle_text, mana_cost)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """
            try:
                self._cursor.execute(query, item)
            except sqlite3.IntegrityError:
                pass

    def get_dataframe_from(self, table):
        query = f"SELECT * FROM {table}"
        return pd.read_sql(query, self._connection)


In [153]:
# Data Objects
with SQLDatabase() as sql_db:
    event_table = sql_db.get_dataframe_from("event")
    deck_table = sql_db.get_dataframe_from("deck")
    deck_list_table = sql_db.get_dataframe_from("deckList")
    card_table = sql_db.get_dataframe_from("card")
    pilot_table = sql_db.get_dataframe_from("pilot")

In [154]:
# Clean Dates
event_table["date"] = pd.to_datetime(event_table["date"], dayfirst=True)
# Renaming ID Columns
event_table.rename(columns={"id": "eventId"}, inplace=True)
deck_table.rename(columns={"id": "deckId"}, inplace=True)
pilot_table.rename(columns={"id": "pilotId"}, inplace=True)

# Adding easier ID column to card_table
card_table["setNumber"] = card_table["setNumber"].str.zfill(3)
card_table["cardId"] = card_table["setNumber"] + card_table["setName"]

In [ ]:
# Replace all the abu codes with updated codes for basic lands
abu_codes = {
    "092abu": (, "Forest"),
    "170abu": (, "Mountain"),
    "130abu": (, "Island"),
    "196abu": (, "Plains"),
    "249abu": (, "Swamp"),
    "065abu": (, "Disenchant"),
    "271abu": (, "Unsummon")
}
# Find remaining codes in deck_list_table that are not in card_table
# Either:
#     Add these sets to the card_table
#     Replace the outdated codes with new codes
other_codes = {
    "171cmd": (, "Scavenging Ooze"),
    "160bng": (, "Temple of Enlightenment"),
    
}

In [120]:
weird_data = deck_list_table.loc[
    ~(deck_list_table["cardId"].isin(card_table["cardId"]))
    & (deck_list_table["cardId"].str.contains("abu"))
]

In [160]:
weird_data_not_abu = deck_list_table.loc[
    ~(deck_list_table["cardId"].isin(card_table["cardId"]))
    & ~(deck_list_table["cardId"].str.contains("abu"))
]

In [161]:
weird_data_not_abu = pd.merge(
    weird_data_not_abu,
    deck_table,
    on="deckId"
)

In [162]:
weird_codes = list(weird_data_not_abu.cardId.unique())

In [163]:
weird_codes

['299eld',
 '171cmd',
 '160bng',
 '228ths',
 '043mor',
 '001frf',
 '360eld',
 '367eld',
 '1136th',
 '119akh',
 '107dar',
 '161bng',
 '356eld',
 '083urs',
 '228roe',
 '292eld',
 '00514c',
 '201isd',
 '245mrd',
 '088ths',
 '096isd',
 '162bng',
 '335eld',
 '364znr',
 '005jou',
 '148rix',
 '225ths',
 '05110m',
 '051m10',
 '246ktk',
 '301m21',
 '165jou',
 '328iko',
 '236ktk',
 '382iko',
 '169inv',
 '065mh1',
 '1276th',
 '048zen',
 '232ktk',
 '235ktk',
 '237ktk',
 '115ori',
 '065som',
 '270eld',
 '227ths',
 '168zen',
 '168m11',
 '109aer',
 '00410m',
 '004m10',
 '255thb',
 '143isd',
 '237urs',
 '255xln',
 '226ths',
 '248xln',
 '164jou',
 '088hou',
 '043hou',
 '224ths',
 '318znr',
 '315znr',
 '018som',
 '036str',
 '316iko',
 '23215m',
 '190m11',
 '229ktk',
 '240ktk',
 '110xln',
 '046mbs',
 '201chk',
 '229ths',
 '190som',
 '079emn',
 '010ktk',
 '040ori',
 '279leg',
 '223soi',
 '082mbs',
 '011dom',
 '147roe',
 '078aer',
 '247ktk',
 '297eld',
 '310ice',
 '0496th',
 '016rav',
 '189dom',
 '10113m',

In [117]:
weird_data_not_abu.loc[weird_data_not_abu.cardId == "160bng"]

,cardId,deckId,count,slot,eventId,pilotId,deckUrl,name,rank
2,160bng,2,3,md,1,4,?e=27938&d=421598&f=ST,UW Control,2
91,160bng,25,1,md,7,26,?e=27886&d=421193&f=ST,UW Control,1
187,160bng,46,3,md,13,46,?e=27941&d=421602&f=ST,Esper Yorion,2
218,160bng,54,4,md,14,53,?e=27876&d=421123&f=ST,UW Yorion,5-8
296,160bng,77,4,md,18,75,?e=27860&d=421016&f=ST,Esper DOOM Yorion,3-4
...,...,...,...,...,...,...,...,...,...
9205,160bng,1832,4,md,348,1260,?e=28178&d=423207&f=ST,Esper DOOM Yorion,5-8
9276,160bng,1852,4,md,353,984,?e=28191&d=423312&f=ST,Esper DOOM Yorion,2
9367,160bng,1873,4,md,359,321,?e=28233&d=423585&f=ST,Bant Yorion,1
9486,160bng,1901,2,md,363,1289,?e=28288&d=423956&f=ST,Esper Yorion,1


In [55]:
events_and_decks = pd.merge(
    event_table, 
    deck_table, 
    on="eventId",
    suffixes=["_event", "_deck"]
)

events_decks_pilots = pd.merge(
    events_and_decks,
    pilot_table,
    on="pilotId",
    suffixes=[None, "_pilot"]
)

events_decks_pilots_deck_lists = pd.merge(
    events_decks_pilots,
    deck_list_table,
    on="deckId",
    suffixes=[None, "_decklist"]
)

full_table = pd.merge(
    events_decks_pilots_deck_lists,
    card_table,
    on=card_table[""]
)

In [123]:
events_decks_pilots_deck_lists

,eventId,name_event,link,date,deckId,pilotId,deckUrl,name_deck,rank,firstName,lastName,cardId,count,slot
0,1,FNM @ MTG Arena Campania,https://www.mtgtop8.com/event?e=27938&f=ST,30/10/20,1,3,?e=27938&d=421597&f=ST,Gruul Aggro,1,Antimo,Morlando,261znr,4,md
1,1,FNM @ MTG Arena Campania,https://www.mtgtop8.com/event?e=27938&f=ST,30/10/20,1,3,?e=27938&d=421597&f=ST,Gruul Aggro,1,Antimo,Morlando,244eld,4,md
2,1,FNM @ MTG Arena Campania,https://www.mtgtop8.com/event?e=27938&f=ST,30/10/20,1,3,?e=27938&d=421597&f=ST,Gruul Aggro,1,Antimo,Morlando,092abu,9,md
3,1,FNM @ MTG Arena Campania,https://www.mtgtop8.com/event?e=27938&f=ST,30/10/20,1,3,?e=27938&d=421597&f=ST,Gruul Aggro,1,Antimo,Morlando,170abu,5,md
4,1,FNM @ MTG Arena Campania,https://www.mtgtop8.com/event?e=27938&f=ST,30/10/20,1,3,?e=27938&d=421597&f=ST,Gruul Aggro,1,Antimo,Morlando,115eld,4,md
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51845,365,MTGO Standard Challenge,https://www.mtgtop8.com/event?e=28292&f=ST,23/11/20,1916,1294,?e=28292&d=423980&f=ST,Dimir Mill,8,pokerswizard,,05110m,1,sb
51846,365,MTGO Standard Challenge,https://www.mtgtop8.com/event?e=28292&f=ST,23/11/20,1916,1294,?e=28292&d=423980&f=ST,Dimir Mill,8,pokerswizard,,066znr,1,sb
51847,365,MTGO Standard Challenge,https://www.mtgtop8.com/event?e=28292&f=ST,23/11/20,1916,1294,?e=28292&d=423980&f=ST,Dimir Mill,8,pokerswizard,,058eld,4,sb
51848,365,MTGO Standard Challenge,https://www.mtgtop8.com/event?e=28292&f=ST,23/11/20,1916,1294,?e=28292&d=423980&f=ST,Dimir Mill,8,pokerswizard,,043mor,2,sb


In [58]:
events_decks_pilots_deck_lists.loc[
    (events_decks_pilots_deck_lists["cardId"].str.contains("m11"))
    & ~(events_decks_pilots_deck_lists["cardId"].str.contains("168"))
]

,eventId,name_event,link,date,deckId,pilotId,deckUrl,name_deck,rank,firstName,lastName,cardId,count,slot
2266,49,Gentry ! @ The Gentry Magic League,https://www.mtgtop8.com/event?e=27808&f=ST,20/10/20,283,40,?e=27808&d=420660&f=ST,Sultai Aggro,1,Karl,Dewhirst-lister,190m11,2,sb
14448,186,Saturday Showdown @ Top Deck Keep,https://www.mtgtop8.com/event?e=27396&f=ST,19/09/20,929,297,?e=27396&d=417383&f=ST,Omnath Ramp,3-4,Fernando,Ortiz,190m11,2,sb
23776,109,Gentry ! @The Jankyard,https://www.mtgtop8.com/event?e=27592&f=ST,4/10/20,601,511,?e=27592&d=419257&f=ST,,1,Jt,Small,190m11,2,sb
31730,173,Gentry ! @ The Gentry Magic League,https://www.mtgtop8.com/event?e=27445&f=ST,22/09/20,864,691,?e=27445&d=417987&f=ST,Gruul Aggro,3-4,Howen,King,190m11,1,sb


In [62]:
events_decks_pilots_deck_lists.loc[
    (events_decks_pilots_deck_lists["cardId"].str.contains("m11"))
]

,eventId,name_event,link,date,deckId,pilotId,deckUrl,name_deck,rank,firstName,lastName,cardId,count,slot
256,174,SCG Tour Online - Challenge @ Star City Games,https://www.mtgtop8.com/event?e=27412&f=ST,21/09/20,871,8,?e=27412&d=417574&f=ST,Omnath Ramp,5-8,Tomohiro,Tsuda,168m11,2,md
273,280,Japan Championship 2020 Winter Weekly Challeng...,https://www.mtgtop8.com/event?e=28065&f=ST,08/11/20,1353,8,?e=28065&d=422441&f=ST,Temur Ultimatum,3-4,Tomohiro,Tsuda,168m11,4,md
354,323,Japan Championship 2020 Winter Daily Trial @ B...,https://www.mtgtop8.com/event?e=28269&f=ST,24/11/20,1704,9,?e=28269&d=423828&f=ST,Temur Ultimatum,1,Takahiro,Omiya,168m11,2,md
2143,109,Gentry ! @The Jankyard,https://www.mtgtop8.com/event?e=27592&f=ST,4/10/20,602,39,?e=27592&d=419258&f=ST,Golgari Control,2,Teddie,Andersson,168m11,4,md
2266,49,Gentry ! @ The Gentry Magic League,https://www.mtgtop8.com/event?e=27808&f=ST,20/10/20,283,40,?e=27808&d=420660&f=ST,Sultai Aggro,1,Karl,Dewhirst-lister,190m11,2,sb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50238,344,Japan Championship 2020 Winter Daily Trial @ B...,https://www.mtgtop8.com/event?e=28197&f=ST,18/11/20,1806,1234,?e=28197&d=423345&f=ST,Temur Ramp Ultimatum,5-8,Ryohei,Kirino,168m11,4,md
51290,358,MTGO Standard Challenge,https://www.mtgtop8.com/event?e=28216&f=ST,16/11/20,1866,1274,?e=28216&d=423467&f=ST,Temur Ramp Ultimatum,2,bobjackson,,168m11,4,md
51307,358,MTGO Standard Challenge,https://www.mtgtop8.com/event?e=28216&f=ST,16/11/20,1867,1275,?e=28216&d=423468&f=ST,Temur Ramp Ultimatum,3,Rav104,,168m11,4,md
51570,361,Japan Championship 2020 Winter Daily Trial @ B...,https://www.mtgtop8.com/event?e=28286&f=ST,25/11/20,1890,1284,?e=28286&d=423945&f=ST,Temur Ramp Ultimatum,2,Masashiro,Kuroda,168m11,4,md


In [83]:
events_decks_pilots_deck_lists.loc[
    events_decks_pilots_deck_lists["name_deck"] == ""
].groupby("deckId").max()

,eventId,name_event,link,date,pilotId,deckUrl,name_deck,rank,firstName,lastName,cardId,count,slot
deckId,,,,,,,,,,,,,
601,109,Gentry ! @The Jankyard,https://www.mtgtop8.com/event?e=27592&f=ST,4/10/20,511,?e=27592&d=419257&f=ST,,1,Jt,Small,299eld,9,sb
930,186,Saturday Showdown @ Top Deck Keep,https://www.mtgtop8.com/event?e=27396&f=ST,19/09/20,722,?e=27396&d=417382&f=ST,,3-4,Christopher,Iaali,264znr,4,sb
931,186,Saturday Showdown @ Top Deck Keep,https://www.mtgtop8.com/event?e=27396&f=ST,19/09/20,723,?e=27396&d=417385&f=ST,,5-8,Travis,Lauro,270eld,4,sb
932,186,Saturday Showdown @ Top Deck Keep,https://www.mtgtop8.com/event?e=27396&f=ST,19/09/20,724,?e=27396&d=417387&f=ST,,5-8,Wesley,Ogletree,249abu,12,sb
934,186,Saturday Showdown @ Top Deck Keep,https://www.mtgtop8.com/event?e=27396&f=ST,19/09/20,726,?e=27396&d=417386&f=ST,,5-8,James,Johnson,259iko,4,sb


In [87]:
card_table.loc[
    card_table["name"].isin(["Mountain", "Swamp", "Forest", "Island", "Plains"])
]

,setNumber,setName,name,cmc,color,mana_cost,standardLegality,oracle_text,cardId
84,266,eld,Forest,0,G,,legal,({T}: Add {G}.),266eld
119,254,eld,Island,0,U,,legal,({T}: Add {U}.),254eld
151,262,eld,Mountain,0,R,,legal,({T}: Add {R}.),262eld
174,250,eld,Plains,0,W,,legal,({T}: Add {W}.),250eld
233,258,eld,Swamp,0,B,,legal,({T}: Add {B}.),258eld
359,286,thb,Forest,0,G,,legal,({T}: Add {G}.),286thb
396,280,thb,Island,0,U,,legal,({T}: Add {U}.),280thb
419,284,thb,Mountain,0,R,,legal,({T}: Add {R}.),284thb
454,278,thb,Plains,0,W,,legal,({T}: Add {W}.),278thb
502,282,thb,Swamp,0,B,,legal,({T}: Add {B}.),282thb


### QUESTION 1: 

How has color dominance changed over time?